## TODO
* worksheets renaming
* split code into small functions
* Rename Queries
* Arrange Colors, Widths for the Layout
* Freeze Panes
* Add headlines for each region
* Add Subtotals for each region
* Add Total for each year in Totals


In [1]:
import pandas as pd
from pandas.io.json import json_normalize
from simple_salesforce import Salesforce
import openpyxl

from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.utils import get_column_letter

from openpyxl.styles import NamedStyle, Alignment, PatternFill, Font

from openpyxl.styles import colors
from openpyxl.styles import Color

In [1]:
# insert login 

In [3]:
# Dateperiod Generation
dates = pd.date_range(start = '2013-12-01', periods = 157, freq = 'M', name = 'Date')
dates = pd.Series(dates).dt.date

In [4]:
# reading login data from external file
with open(salesforce_login_filepath, 'r') as f:
    username = f.readline().rstrip()
    password = f.readline().rstrip()
    token = f.readline()

# logging in to Salesforce
sf = Salesforce(password=password, username=username, security_token=token)

In [5]:
account = sf.query_all(query_2)
df_account = pd.DataFrame(account['records'])

In [6]:
territory = sf.query_all(query_3)
df_territory = pd.DataFrame(territory['records'])

In [7]:
opp = sf.query_all(query)
df_opp = pd.DataFrame(opp['records'])

In [8]:
acc_ter = pd.merge(df_account, df_territory, how='left', left_on='Territory__c', right_on='Id')
df = pd.merge(df_opp, acc_ter, how='right', left_on='AccountId', right_on='Id_x')

In [9]:
df.drop(['attributes', 'attributes_x', 'Id_x', 'Territory__c', 'attributes_y', 'Id_y'], axis=1, inplace = True)

In [10]:
df.dropna(subset=['AccountId'], inplace = True)

In [11]:
df['Maintenance_Start_Date__c'] = pd.to_datetime(df['Maintenance_Start_Date__c'])

# Workbook styles

In [13]:
top_column_style = NamedStyle(name='Top Column')
top_column_style.font = Font(name='Arial', size=8, bold=True, italic=False, color='FFFFFF')
top_column_style.fill = PatternFill(fgColor='808080', patternType='solid')
top_column_style.alignment = Alignment(horizontal='center')

In [14]:
top_dates_style = NamedStyle(name='Top Dates')
top_dates_style.font = Font(name='Arial', size=8, bold=True, italic=False, color='FFFFFF')
top_dates_style.fill = PatternFill(fgColor='44546A', patternType='solid')
top_dates_style.alignment = Alignment(horizontal='center')
top_dates_style.number_format = "MMM-YY"

In [15]:
general_style = NamedStyle(name='General Style')
general_style.font = Font(name='Arial', size=8, bold=False, italic=False, color='000000')

In [16]:
left_dates_style = NamedStyle(name='Left Dates')
left_dates_style.font = Font(name='Arial', size=8, bold=False, italic=False, color='000000')
left_dates_style.alignment = Alignment(horizontal='center')
left_dates_style.number_format = "MMM-YY"

In [17]:
currency_style = NamedStyle(name='Currencies')
currency_style.font = Font(name='Arial', size=8, bold=False, italic=False, color='000000')
currency_style.number_format = "$#,##0.00"

## Functions declaration

In [18]:
def currency_fix(df):
    if df['CurrencyIsoCode'] == 'USD':
        return df['Amount']
    elif df['CurrencyIsoCode'] == 'EUR':
        return df['Amount']/0.84
    elif df['CurrencyIsoCode'] == 'GBP':
        return df['Amount']/0.75

In [19]:
def maint_term_trim(string_to_trim):
    '''Trims the string of Support_and_Maintenance__c to single character and converts it to integer.
    Then the function multiplies the value in years by 12 months.'''
    if string_to_trim == 'None':
        string_to_trim = 0
        return string_to_trim
    else:
        string_to_trim = string_to_trim[:1]
        string_to_trim = int(string_to_trim)*12
        return string_to_trim

In [20]:
def ps_msp_total_contract(sheet):
    '''Function fills the values for the professional services and MSP bookings.'''
    
    # initialization of starting rows for dates and column titles
    start_row_dates = 17
    start_row_titles = 14
    
    # Loop over the regions available in data
    for i, val in enumerate(df['Country_Group__c'].unique()):
        
        # printing of Country - Build Revenue Title before each regional block
        sheet.cell(row=i+start_row_titles, column=2, value=val).style = general_style
        sheet.cell(row=i+start_row_titles+1, column=2, value='Build Revenue').style = general_style
        
        # printing of the left column dates, number of months in a period. Initialization of start and end date for formula.
        # 12 might be deleted
        for j, date in enumerate(dates):
            sheet.cell(row=j+start_row_dates, column=3, value=dates[j]).style = left_dates_style
            sheet.cell(row=j+start_row_dates+1, column=5, value=12).style = general_style
            start_date = sheet.cell(row=j+start_row_dates,column=3).value
            end_date = sheet.cell(row=j+start_row_dates+1,column=3).value
            
            # df filtering and pushing of the data to a cell based on worksheet
            if sheet.title == 'ps':
                push_df = df[(df['Country_Group__c']==val)&(df['Type']!='MSP')&(df['Maintenance_Term']==0)&(df['CloseDate']>start_date)&(df['CloseDate']<=end_date)]
                sheet.cell(row=j+start_row_dates+1, column=4, value=push_df['Bookings'].sum()).style = currency_style
                sheet.cell(row=j+start_row_dates+1, column=j+7, value=push_df['Bookings'].sum()).style = currency_style
                push_df = df[(df['Type']!='MSP')&(df['Maintenance_Term']==0)&(df['CloseDate']>start_date)&(df['CloseDate']<=end_date)]
                sheet.cell(row=10, column=j+7, value = push_df['Bookings'].sum()).style = currency_style
            elif sheet.title == 'msp':
                push_df = df[(df['Country_Group__c']==val)&(df['Type']=='MSP')&(df['CloseDate']>start_date)&(df['CloseDate']<=end_date)]
                sheet.cell(row=j+start_row_dates+1, column=4, value=push_df['Bookings'].sum()).style = currency_style
                sheet.cell(row=j+start_row_dates+1, column=j+7, value=push_df['Bookings'].sum()).style = currency_style
                push_df = df[(df['Type']=='MSP')&(df['CloseDate']>start_date)&(df['CloseDate']<=end_date)]
                sheet.cell(row=10, column=j+7, value = push_df['Bookings'].sum()).style = currency_style
            
            elif sheet.title == 'normal' | sheet.title == 'addon':
                push_df = df[(df['Type']=='MSP')&(df['CloseDate']>start_date)&(df['CloseDate']<=end_date)]
                sheet.cell(row=10, column=j+10, value = push_df['Bookings'].sum()).style = currency_style
                
            elif sheet.title == 'sub':
                push_df = df[(df['Type']=='MSP')&(df['CloseDate']>start_date)&(df['CloseDate']<=end_date)]
                sheet.cell(row=10, column=j+8, value = push_df['Bookings'].sum()).style = currency_style
            
            elif sheet.title == 'renewal':
                push_df = df[(df['Type']=='MSP')&(df['CloseDate']>start_date)&(df['CloseDate']<=end_date)]
                sheet.cell(row=10, column=j+8, value = push_df['Bookings'].sum()).style = currency_style
            
        start_row_dates = (len(dates)+start_row_dates)+6
        start_row_titles = (len(dates)+start_row_titles)+5

In [21]:
def renewal_total_contract(sheet):
    '''Function fills the values for renewals.'''
    # initialization of starting rows for dates and column titles
    start_row_dates = 17
    start_row_titles = 13
    start_row_term = 15
    
    for i, country in enumerate(df['Country_Group__c'].unique()):
        # printing of Country - Build Revenue Title before each regional block
        sheet.cell(row=start_row_titles, column=2, value=country).style = general_style
        sheet.cell(row=start_row_titles+1, column=2, value='Build Revenue').style = general_style
        
        term_df = df[(df['Country_Group__c']==country)&(df['Type']!='MSP')&(df['Maintenance_Term']!=0)&(df['QuoteType__c']=='Renewal')]
        for k, term in enumerate(sorted(term_df['Maintenance_Term'].unique())):
            sheet.cell(row=start_row_term, column=2, value=str(int(term/12)) + ' Year Deferral').style = general_style
            start_row_titles = (len(dates)+start_row_dates)+2
            z_offset = 0
            
            for j, date in enumerate(dates):
                sheet.cell(row=j+start_row_dates, column=3, value=dates[j]).style = left_dates_style
                sheet.cell(row=j+start_row_dates+1, column=6, value=term).style = general_style
                start_date = sheet.cell(row=j+start_row_dates,column=3).value
                end_date = sheet.cell(row=j+start_row_dates+1,column=3).value
                # filling total contract
                push_df = df[(df['Country_Group__c']==country)&(df['QuoteType__c']=='Renewal')&(df['Type']!='MSP')&(df['Maintenance_Term']==term)&(df['Maintenance_Start_Date__c']>start_date)&(df['Maintenance_Start_Date__c']<=end_date)]
                sheet.cell(row=j+start_row_dates+1, column=5, value= push_df['Bookings'].sum()).style = currency_style
                # filling monthly revenue
                sheet.cell(row=j+start_row_dates+1, column=7, value=
                          (sheet.cell(row=j+start_row_dates+1, column=5).value)
                           /
                           (sheet.cell(row=j+start_row_dates+1, column=6).value)).style = currency_style
                
                # filling ACV
                sheet.cell(row=j+start_row_dates+1, column=4, value=
                          (sheet.cell(row=j+start_row_dates+1, column=7).value) * 12).style = currency_style
                
                # filling revenue waterfall
                if start_date == sheet.cell(row=3, column=j+8).value:
                    for z in range(term):
                        sheet.cell(row=j+start_row_dates, column=z+9+z_offset, value=
                                    sheet.cell(row=j+start_row_dates, column=7).value
                                    ).style = currency_style
                    z_offset += 1
                
            start_row_dates = (len(dates)+start_row_dates)+6
            start_row_term = (len(dates)+start_row_term)+6


In [ ]:
def normal_total_contract(sheet):
    '''Function fills the values for renewals.'''
    # initialization of starting rows for dates and column titles
    start_row_dates = 17
    start_row_titles = 13
    start_row_term = 15
    
    for i, country in enumerate(df['Country_Group__c'].unique()):
        # printing of Country - Build Revenue Title before each regional block
        sheet.cell(row=start_row_titles, column=2, value=country).style = general_style
        sheet.cell(row=start_row_titles+1, column=2, value='Build Revenue').style = general_style
        
        term_df = df[(df['Country_Group__c']==country)&(df['Type']!='MSP')&(df['Maintenance_Term']!=0)&(df['QuoteType__c']=='Normal')]
        for k, term in enumerate(sorted(term_df['Maintenance_Term'].unique())):
            sheet.cell(row=start_row_term, column=2, value=str(int(term/12)) + ' Year Deferral').style = general_style
            start_row_titles = (len(dates)+start_row_dates)+2
            z_offset = 0
            
            for j, date in enumerate(dates):
                sheet.cell(row=j+start_row_dates, column=3, value=dates[j]).style = left_dates_style
                sheet.cell(row=j+start_row_dates+1, column=7, value=term).style = general_style
                start_date = sheet.cell(row=j+start_row_dates,column=3).value
                end_date = sheet.cell(row=j+start_row_dates+1,column=3).value
                # filling total contract
                push_df = df[(df['Country_Group__c']==country)&(df['QuoteType__c']=='Normal')&(df['Type']!='MSP')&(df['Maintenance_Term']==term)&(df['CloseDate']>start_date)&(df['CloseDate']<=end_date)]
                sheet.cell(row=j+start_row_dates+1, column=6, value= push_df['Bookings'].sum()).style = currency_style
                # filling monthly revenue
                sheet.cell(row=j+start_row_dates+1, column=8, value=
                          (sheet.cell(row=j+start_row_dates+1, column=6).value*0.25)
                           /
                           (sheet.cell(row=j+start_row_dates+1, column=7).value)).style = currency_style
                
                # filling ACV
                sheet.cell(row=j+start_row_dates+1, column=4, value=
                          ((sheet.cell(row=j+start_row_dates+1, column=6).value) * 0.75) +
                           ((sheet.cell(row=j+start_row_dates+1, column=8).value) * 12)).style = currency_style
                
                # filling Def. Maintenance
                sheet.cell(row=j+start_row_dates+1, column=5, value=
                          (sheet.cell(row=j+start_row_dates+1, column=6).value)*0.25).style = currency_style
                
                # filling revenue waterfall
                if start_date == sheet.cell(row=3, column=j+9).value:
                    for z in range(term):
                        if z == 0:
                            sheet.cell(row=j+start_row_dates, column=z+10+z_offset, value=
                                    sheet.cell(row=j+start_row_dates, column=8).value + 
                                    (sheet.cell(row=j+start_row_dates, column=6).value * 0.75)
                                    ).style = currency_style
                        else:
                            sheet.cell(row=j+start_row_dates, column=z+10+z_offset, value=
                                        sheet.cell(row=j+start_row_dates, column=8).value
                                        ).style = currency_style
                    z_offset += 1
                
            start_row_dates = (len(dates)+start_row_dates)+6
            start_row_term = (len(dates)+start_row_term)+6

In [23]:
def top_column_style_generator(list):
    # generation of top columns
    for i, col in enumerate(list):
        sheet.cell(row=3, column=i+3, value=col).style = top_column_style
        k = get_column_letter(i+3)
        sheet.column_dimensions[k].width = 25
    
    # generation of revenue sources column on top
    for i, row in enumerate(revenue_column):
        sheet.cell(row=i+5, column=len(list)+2,value=row).style = general_style
    
    # generation of total datarange on top
    for i, col in enumerate(dates[1:]):
        sheet.cell(row=3, column=len(list)+4+i, value=col).style = top_dates_style
        k = get_column_letter(i+len(list)+4)
        sheet.column_dimensions[k].width = 12

# Data Cleaning

In [24]:
df['Bookings'] = df.apply(currency_fix, axis = 1)

In [25]:
# Maintenance term is converted to the number of months.
df['Maintenance_Term'] = df['Support_and_Maintenance__c'].apply(maint_term_trim)
df.drop(['Support_and_Maintenance__c', 'Amount'], axis = 1, inplace = True)

In [26]:
df['Country_Group__c'] = df['Country_Group__c'].replace(['Russia & CIS', 'Benelux & Nordics', 'Italy & Iberia', 'UKI',
                                                        'DACH', 'Middle East', 'North Africa','South Africa', 'Central Africa', 
                                                         'East Europe', 'France'], 'EMEA')
df['Country_Group__c'] = df['Country_Group__c'].replace('NAM', 'North America')
df['Country_Group__c'] = df['Country_Group__c'].replace(['JAPAC', 'India'], 'APAC')
df['Country_Group__c'] = df['Country_Group__c'].replace('LATAM', 'ROW')
df['Country_Group__c'] = df['Country_Group__c'].fillna('ROW')

In [27]:
df['CloseDate'] = pd.to_datetime(df['CloseDate'])
df['CloseDate'] = df['CloseDate']

In [28]:
df = df.drop(df[df['Bookings']==0].index)

## Workbook manipulation


In [29]:
wb = openpyxl.Workbook()
raw_data = wb.active
raw_data.title = 'Raw Data'
raw_data.sheet_properties.tabColor = 'D3D3D3'

In [30]:
# assign named styles to workbook
wb.add_named_style(top_column_style)

In [31]:
for r in dataframe_to_rows(df, index=True, header=True):
    raw_data.append(r)

for cell in raw_data['A'] + raw_data[1]:
    cell.style = 'Pandas'

In [32]:
# creating sheets
sheet_names = ['Total RRS', 'Normal RRS', 'Add-on RRS', 'Normal', 'Renewal', 'Add-on', 'Professional Services', 'MSP', 'Subscription', 'Normal S&M', 'Add-on S&M']
sheets = ['t_rrs', 'n_rrs', 'ad_rrs', 'total', 'normal', 'renewal', 'addon', 'ps', 'msp', 'sub', 'n_sm', 'ad_sm']
for i, sheet in enumerate(sheets):
    sheets[i] = wb.create_sheet(sheet, i)

# Layout

In [ ]:
# generation of top-left titles
top_columns = ['Close Month', 'ACV', 'Def.Maintenance', 'Total Contract', 'Term in Months', 'Monthly Revenue']
revenue_column = ['Normal', 'Renewal', 'Add-on Purchase', 'Subscription', 'MSP', 'Professional Services', 'Total Sales Build Revenue']

for sheet in sheets:
    if sheet.title=='normal' or sheet.title == 'addon':
        top_column_style_generator(top_columns)
        if sheet.title=='normal':
            normal_total_contract(sheet)
            
    elif sheet.title=='renewal' or sheet.title=='ad_rrs' or sheet.title=='n_rrs':
        top_column_style_generator(top_columns[:2] + top_columns[3:])
        renewal_total_contract(sheet)
            
    elif sheet.title=='ps' or sheet.title=='msp':
        top_column_style_generator(top_columns[0:1]+top_columns[3:5])
        ps_msp_total_contract(sheet)

            
    elif sheet.title=='sub':
        top_column_style_generator(top_columns[0:1]+top_columns[3:])

            
    elif sheet.title=='n_sm' or sheet.title=='ad_sm':
        top_column_style_generator(top_columns[0:1] + top_columns[2:])

    


In [ ]:
# wb.save(r"Python\Revenue Waterfall\temp.xlsx")